In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, CuDNNLSTM, Dense, LSTM
import pandas as pd
import re
import numpy as np
import unicodedata
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD


nlp = spacy.load("pt_core_news_sm")

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Tweets_Mg.csv', header=None)
df.columns = ['texto', 'sentimento']
df.head()

,texto,sentimento
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,��� https://t.co/BnDsO34qK0,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [3]:
def preprocess(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in STOP_WORDS])
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[\r|\n|\r\n]+', ' ', text)
    text = re.sub(r'[?!.,@#:]', '', text)
    special_char_pattern = re.compile(r'[\}\}\\\(\)\./!-]')
    text = special_char_pattern.sub(" ", text)  
    
    return text.strip()

In [4]:
df['texto'] = df['texto'].apply(preprocess)
df['sentimento'] = pd.factorize(df.sentimento)[0]
df.head()

,texto,sentimento
0,catedral santo antonio governador valadares ...,0
1,governador valadares minas gerais https tco b...,0
2,governador valadares minas gerais https tco d...,0
3,https tco bndso34qk0,0
4,psol questionar aumento vereadores e prefeito ...,1


In [5]:
tokenizer = Tokenizer(num_words=None, lower=True, split=' ', filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(df['texto'])
X = tokenizer.texts_to_sequences(df['texto'])
X = pad_sequences(X)

In [6]:
embed_size = 150
lstm_out = 400
batch_size = 64

model = Sequential()
model .add(Embedding(4000, embed_size, input_length = X.shape[1]))
model .add(LSTM(lstm_out))
model .add(Dense(3, activation='softmax'))

modelCUDA = Sequential()
modelCUDA .add(Embedding(4000, embed_size, input_length = X.shape[1]))
modelCUDA .add(CuDNNLSTM(lstm_out))
modelCUDA .add(Dense(3, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
modelCUDA.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print(model.summary())
print('\n***\n')
print(modelCUDA.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 150)           600000    
_________________________________________________________________
lstm (LSTM)                  (None, 400)               881600    
_________________________________________________________________
dense (Dense)                (None, 3)                 1203      
Total params: 1,482,803
Trainable params: 1,482,803
Non-trainable params: 0
_________________________________________________________________
None

***

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 26, 150)           600000    
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 400)               883200    
_______________________________________________________

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, pd.get_dummies(df['sentimento']), test_size=0.3,random_state=42)

In [8]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=50)

/home/gabriel/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
5739/5739 [==============================] - 6s 1ms/step - loss: 1.0871 - acc: 0.4032
Epoch 2/50
5739/5739 [==============================] - 5s 884us/step - loss: 1.0767 - acc: 0.4043
Epoch 3/50
5739/5739 [==============================] - 5s 884us/step - loss: 1.0566 - acc: 0.4370
Epoch 4/50
5739/5739 [==============================] - 5s 887us/step - loss: 0.9556 - acc: 0.5496
Epoch 5/50
5739/5739 [==============================] - 5s 886us/step - loss: 0.7188 - acc: 0.7076
Epoch 6/50
5739/5739 [==============================] - 5s 887us/step - loss: 0.4420 - acc: 0.8353
Epoch 7/50
5739/5739 [==============================] - 5s 887us/step - loss: 0.3215 - acc: 0.8899
Epoch 8/50
5739/5739 [==============================] - 5s 886us/step - loss: 0.4197 - acc: 0.8472
Epoch 9/50
5739/5739 [==============================] - 5s 887us/step - loss: 0.2280 - acc: 0.9310
Epoch 10/50
5739/5739 [==============================] - 5s 912us/step - loss: 0.1919 - acc: 0.9418
Epoch 11/50

In [9]:
modelCUDA.fit(X_train, y_train, batch_size=batch_size, epochs=50)

/home/gabriel/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
5739/5739 [==============================] - 7s 1ms/step - loss: 1.0850 - acc: 0.4006
Epoch 2/50
5739/5739 [==============================] - 1s 208us/step - loss: 1.0734 - acc: 0.4043
Epoch 3/50
5739/5739 [==============================] - 1s 212us/step - loss: 1.0541 - acc: 0.4219
Epoch 4/50
5739/5739 [==============================] - 1s 211us/step - loss: 0.9775 - acc: 0.5374
Epoch 5/50
5739/5739 [==============================] - 1s 216us/step - loss: 0.7535 - acc: 0.6844
Epoch 6/50
5739/5739 [==============================] - 1s 209us/step - loss: 0.4927 - acc: 0.8146
Epoch 7/50
5739/5739 [==============================] - 1s 208us/step - loss: 0.3725 - acc: 0.8691
Epoch 8/50
5739/5739 [==============================] - 1s 210us/step - loss: 0.5173 - acc: 0.8064
Epoch 9/50
5739/5739 [==============================] - 1s 210us/step - loss: 0.3466 - acc: 0.8848
Epoch 10/50
5739/5739 [==============================] - 1s 213us/step - loss: 0.2338 - acc: 0.9251
Epoch 11/50

In [10]:
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('score: {0:.2}'.format(score))
print('acc: {0:.2}'.format(acc))

2460/2460 [==============================] - 1s 324us/step
score: 0.89
acc: 0.6


In [11]:
def deprocess(text):
    text_test = preprocess(text)
    text_test = tokenizer.texts_to_sequences([text])
    text_test = pad_sequences(text_test)
    text_to_array = np.zeros(X.shape[1])
    text_to_array[(text_to_array.shape[0] - text_test.shape[1]):] = text_test
    result = np.argmax(model.predict(text_to_array.reshape(1,X.shape[1])))

    if result == 0:
        print('Neutro')
    elif result == 1:
        print('Negativo')
    else:
        print('Positvo')

In [12]:
deprocess('PM apreende maconha e celular no presidio')

Neutro
